Что делать?

 * Датасет ml-latest
 * Вспомнить подходы, которые мы разбирали
 * Выбрать понравившийся подход к гибридным системам
 * Написать свою
 * Материалы здесь.

In [78]:
from surprise import SVD,KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import NearestNeighbors
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np

In [129]:
# для новых пользователей (нет ни одного просмотра) выведем 10 фильмов с наивысшей средней оценкой
# делаем смешивание, если были просмотры:
##   находим по 100 наилучших фильмов по content-based (на основе пяти просмотренных фильмов с наивысшей оценкой)
##   и по 100 наилучших фильмов по коллаборативной фильтрации 
##   и смешиваем результаты (получаем 200 фильмов)
# по этим 200 фильмам - предсказываем оценку по SVD и выводим первые 10 (каскад)
films_for_new_user = 10
best_films = 5
films_CB = 100
films_collaborative = 100
final_films = 10

In [125]:
user_id = 15.0

In [4]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True) #очищаем от пустот 

In [6]:
def change_genre(str_):
    return ' '.join(str_.replace(' ','').replace('-', '').split('|'))

In [22]:
# датасет для сюрпрайза
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating})

In [23]:
min_ = dataset.rating.min()
max_ = dataset.rating.max()
reader = Reader(rating_scale=(min_, max_))
data = Dataset.load_from_df(dataset, reader)

In [24]:
trainset, testset = train_test_split(data, test_size=.15)

In [135]:
def recommendation(user_id):
    user_films = movies_with_ratings[movies_with_ratings.userId == user_id].title.unique()
    if len(user_films) == 0:
        # 1. Найдем 10 фильмов с наивысшей оценкой
        top_films = movies_with_ratings.groupby('title')[['rating']].mean().sort_values('rating', ascending=False).head(films_for_new_user)
        print(top_films)
    else:
        # 2. Строим content-based 
        user_movies = movies_with_ratings[movies_with_ratings.userId == user_id]
        user_movies_unique = user_movies.title.unique()
    #     best_genre = user_movies.loc[user_movies.rating.idxmax()]['genres'] #найдем жанр с максимальной оценкой пользователя
        high_rating_films = user_movies.sort_values('rating')[-(best_films):] #найдем жанры первых 5-ти фильмов пользователя с макс оценкой
        best_genres = ' '.join([change_genre(g) for g in high_rating_films.genres.values])
        rec_movies_with_ratings = movies_with_ratings[movies_with_ratings.userId != user_id] #фильмы, которые пользователь не смотрел
        rec_movies = movies[movies.movieId.isin(rec_movies_with_ratings.movieId.unique())] #фильмы, которые пользователь не смотрел
        rec_movie_genres = [change_genre(g) for g in rec_movies.genres.values]

        count_vect_cb = CountVectorizer()
        X_train_counts_cb = count_vect_cb.fit_transform(rec_movie_genres)
        tfidf_transformer_cb = TfidfTransformer()
        X_train_tfidf_cb = tfidf_transformer_cb.fit_transform(X_train_counts_cb)
        neigh_cb = NearestNeighbors(n_neighbors = films_CB, n_jobs=-1, metric='euclidean')
        neigh_cb.fit(X_train_tfidf_cb)
        predict_cb = count_vect_cb.transform([best_genres])
        X_tfidf2_cb = tfidf_transformer_cb.transform(predict_cb)
        res_cb = neigh_cb.kneighbors(X_tfidf2_cb, return_distance=True)
        res_cb2 = rec_movies.iloc[res_cb[1][0]] 
        # 3. Строим коллаб.фильтрацию
        algo_coll = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'content_based': False})
        algo_coll.fit(trainset)
        test_pred = algo_coll.test(testset)
        recommend = {}
        for t in rec_movies.title.values:
            est = algo_coll.predict(uid=user_id, iid=t).est
            recommend[t] = est
        res_col = sorted(recommend.items(), key=lambda t: t[1], reverse=True)[:(films_collaborative)]
        # 4. находим фильмы, рекомендуемые обоими методами (смешивание)
        res_mix = []
        for i in res_col:
            res_mix.append(i[0])
        for j in res_cb2.title.values:
            if j in res_mix:
                continue
            else:
                res_mix.append(j)
        # 4. предсказываем оценку (SVD) и выводим лучшие 10 (каскад)
        algo_svd = SVD(n_epochs = 5, n_factors = 20)
        algo_svd.fit(trainset)
        predictions_svd = algo_svd.test(testset)   
        scores = []
        titles = []
        for movie in res_mix:    
            scores.append(algo_svd.predict(uid = user_id, iid = movie).est) #как бы этот фильм (который он не смотрел) оценил бы пользователь
            titles.append(movie)
        best_indexes = np.argsort(scores)[-(final_films):] #сортируем
        for i in reversed(best_indexes):
            print(titles[i],scores[i])

In [136]:
recommendation(user_id)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Star Wars: Episode IV - A New Hope (1977) 3.839337146253773
Star Wars: Episode VI - Return of the Jedi (1983) 3.8230015203334986
Star Wars: Episode V - The Empire Strikes Back (1980) 3.810771338965113
Road Warrior, The (Mad Max 2) (1981) 3.619356867245125
The Martian (2015) 3.581358467867611
2001: A Space Odyssey (1968) 3.566148375697205
Serenity (2005) 3.541685854767829
Star Trek: First Contact (1996) 3.5187302890468293
Children of Men (2006) 3.5167769031816047
Fifth Element, The (1997) 3.505629193042564
